<a href="https://colab.research.google.com/github/diefgallegorio/Prueba_Facturedo/blob/main/Prueba_ingreso_Factoredo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Facturedo - Prueba básica de Python**

# Presentado por: [Diego Gallego Ríos](https://www.linkedin.com/in/diego-gallego-r%C3%ADos-18218b133/)



---


*Para visualizar de manera correcta la ejecución del notebook, abra el archivo en Google Colab, mediante el botón que está en la parte superior del archivo en Github. Así mismo  o descargando el archivo pynb, podrá ejecutar el código nuevamente para verificar su funcionamiento.*

In [1]:
import pandas as pd
import whois
import folium
import unicodedata
import string

from geopy.exc import GeocoderTimedOut, GeocoderQuotaExceeded
from geopy.geocoders import Nominatim
from validate_email import validate_email

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials



---


### **CASO 1**
Escriba un método palindromo(frase) que recibe una frase, palabra o número y devuelve si esta es palíndromo o no. Luego, escriba un programa principal que solicita al usuario una palabra, envía la palabra al método anterior y luego le dice al usuario si es palíndromo. Tenga en cuenta que la palabra o frase, puede incluir espacios, puntos y comas.

In [2]:
class Caso1:
    def __init__ (self):
        self.frase = input("Escriba una frase, palabra o número: ")

    def estandarizar_frase(self):
        """
        Función que elimina acentos, espacios, signos de puntuación y
        se transforma toda la frase a minúscula para facilitar su uso.
        """
        frase = self.frase

        frase = unicodedata.normalize("NFD", str(frase))
        sin_acentos = frase.encode("utf8").decode("ascii", "ignore")
        minuscula = str(sin_acentos).lower()
        sin_espacios = minuscula.replace(" ", "")
        frase_estandar = sin_espacios.translate(str.maketrans('', '',
                                                string.punctuation))

        return frase_estandar

    def palindrome(self):
        """
        Función que recibe una frase e imprime si es o no es palindrome.
        """
        frase = self.frase

        # Se estandariza la frase, para eliminar acentos, espacios y
        # signos de puntuación.
        frase_estandar = self.estandarizar_frase()

        # Comparo si la frase es igual que la frase al revés e imprimo el
        # resultado de esta comparación

        if(frase_estandar == frase_estandar[::-1]):
            print('\n"' + frase + '" Es Palindrome')
        else:
            print('\n"' + frase + '" No es Palindrome')


frase = Caso1()  
frase.palindrome()
    


Escriba una frase, palabra o número: Allí va Ramón y no maravilla.

"Allí va Ramón y no maravilla." Es Palindrome



---


### **Caso 2:**

Cree un script en python que lea estas [direcciones](https://drive.google.com/file/d/1s1tsEppbFTwcJEAB78QoF3xLaKFAa_8S/view) (use la columna ADDRESS), las convierta a latitud y longitud y luego se visualicen en un mapa.

In [3]:
# Para la lectura del archivo alojado en Google Drive, se crea un cliente 
# pydrive con su autenticación correspondiente.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Se lee el archivo mediante el id y se descarga en el entorno
file_id = '1s1tsEppbFTwcJEAB78QoF3xLaKFAa_8S'
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile('addresses_sweden.csv') 

# Se lee el archivo donde están las direcciones mencionadas.
addresses = pd.read_csv("addresses_sweden.csv")

# Se inicializa nominatim, el cual es útil al momento de determinar
# las coodenadas que se graficarán en el mapa.
locator = Nominatim(user_agent="pruebaFacturedo")

# Con geocode() busco las direcciones que están en la columna 'ADDRESS'
# y las asigno en una nueva columna 'location'
addresses['location'] = addresses['ADDRESS'].apply(
                        lambda x: locator.geocode(x))

# En location_none se guardan las direcciones que no fueron posibles
# encontrar y posteriormente se borran del DataFrame original
location_none = addresses[addresses['location'].isna()]['ADDRESS']
addresses.dropna(subset=['location'], inplace=True)

# Se crea la columna point, la cual contiene las coordenadas.
addresses['point'] = addresses['location'].apply(lambda x: tuple(x.point))

# Se separa la columna point en las columnas:
# 'latitude', 'longitude' y 'altitude'.
addresses[['latitude', 'longitude', 'altitude']] = pd.DataFrame(
    addresses['point'].tolist(), index=addresses.index)

# Si hubo direcciones no encontradas, se imprimen notifica la cantidad
# y se imprimen dichas direcciones.
if(len(location_none) != 0):
    print('No se pudieron graficar las siguientes ' +
          str(len(location_none)) +
          ' ubicaciones')
    print(location_none.to_string(index=False))


# Se inicializa el mapa donde se visualizarán los puntos
mapa = folium.Map(
    location=[addresses['latitude'].mean(), addresses['longitude'].mean()],
    tiles='Stamen Terrain',
    zoom_start=13,
    )

# Se dibujan los puntos del mapa
addresses.apply(lambda row: folium.CircleMarker
                (location=[row["latitude"], row["longitude"]]).add_to(mapa),
                axis=1)

# Se imprime el mapa con las direcciones del archivo
mapa


No se pudieron graficar las siguientes 2 ubicaciones
 Rålambsvägen 7-9,112 59,STOCKHOLM,Stockholms l...
 Liljeholmstorget 3, plan 1,117 63,STOCKHOLM,St...



---

### **Caso 3:** 
Escriba un programa que recibe un correo electrónico (e.j. francy.camacho@facturedo.com), toma el dominio y da como salida la fecha de creación o de registro en whois (en python, existe una librería para ello).

In [4]:
def fecha_creacion_dominio(dominio):
    """
    Función que retorna la fecha de creación (De acuerdo a Whois) de un
    dominio, en caso de no encontrar fecha de creación devuelve None.
    """
    try:
        fecha_creacion = whois.whois(dominio).creation_date
        return fecha_creacion
    except:
        print('\nEl dominio "' + dominio + '" no ha sido asignado.')
        return None


def imprimir_fecha(fecha):
    """
    Función que imprime una o muchas fechas, de acuerdo a lo que le ingrese
    a la función.
    """

    # Se evalúa si se obtuvo más de una fecha de creación, ya que de acuerdo a
    # esto variará el modo de impresión de la respuesta.
    if(type(fecha) == list):
        print('El dominio "' + dominio + '" tiene ' + str(len(fecha)) +
              " fechas de creación:")
        for i in range(len(fecha)):
            print(str(i + 1) + " ➔ " + str(fecha[i]))

    else:
        print('\nEl dominio "' + dominio +
              '" tiene la siguiente fecha de creación:\n' + str(fecha))


# Se inicializa la variable donde se alojará el correo electrónico
# y valido que el usuario ingrese un email válido.

email = ''
while(email == ''):
    email = input("Escriba el correo electrónico: ")
    if(validate_email(email)):

        # Se separa lo que está posterior al '@' para obtener el dominio.
        dominio = email.split("@")[1]

        # Se obtiene la(s) fecha(s) de creación del dominio
        fecha = fecha_creacion_dominio(dominio)

        # Se valida nuevamente que exista alguna fecha de creación del dominio.
        if(fecha is None):
            email = ''
            print("¡ Error, correo no válido !\n")
        else:
            imprimir_fecha(fecha)
    else:
        email = ''
        print("¡ Error, correo no válido !\n")


Escriba el correo electrónico: francy.camacho@facturedo.com

El dominio "facturedo.com" tiene la siguiente fecha de creación:
2014-09-12 03:56:35
